In [1]:
import os
import pandas as pd

directory_path = '/workspace/outputDir'

dataframes = []

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)

combined_dataset = pd.concat(dataframes, ignore_index=True)


print("Combined dataset created successfully!")

Combined dataset created successfully!


In [2]:
len(combined_dataset)

44272

In [1]:
import os
from PIL import Image
import numpy as np
import torch

In [2]:
def getSize(imagePath):
    image = Image.open(imagePath)

    image = image.convert('L')

    image_array = np.array(image)
    
    image_array = 255 - image_array
    
    dense_tensor = torch.tensor(image_array, dtype=torch.uint8)

    # Find the indices of non-zero elements
    non_zero_indices = torch.nonzero(dense_tensor, as_tuple=True)
    
    # Get the values of the non-zero elements
    non_zero_values = dense_tensor[non_zero_indices]

    # Create a sparse tensor
    sparse_tensor = torch.sparse_coo_tensor(
        indices=torch.stack(non_zero_indices),
        values=non_zero_values,
        size=dense_tensor.shape
    )

    sparse_tensor = sparse_tensor.coalesce()
    
    indices_memory = sparse_tensor.indices().numel() * sparse_tensor.indices().element_size()
    values_memory = sparse_tensor.values().numel() * sparse_tensor.values().element_size()
    sparse_tensor_memory_bytes = indices_memory + values_memory
    sparse_tensor_memory_mb = sparse_tensor_memory_bytes / (1024 ** 2)
    
    return sparse_tensor, sparse_tensor_memory_mb

    # print(image_array.shape) 

In [3]:
combined_dataset['plane0_sparse_tensor'] = None
combined_dataset['plane1_sparse_tensor'] = None
combined_dataset['plane2_sparse_tensor'] = None

In [4]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 850.1 kB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 28.4 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 5.4 MB/s eta 0:00:0078.4 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import pandas as pd
import pickle

directory_path = '/workspace/outputDir'

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # Add new columns for sparse tensors
        df['plane0_sparse_tensor'] = None
        df['plane1_sparse_tensor'] = None
        df['plane2_sparse_tensor'] = None

        sizeFull = 0

        for i in range(len(df)):
            if i % 100 == 0:
                print(f"Processing file: {filename}, iteration: {i}")
                print(sizeFull)
                print("====================")
            for imName in ['plane1_file', 'plane2_file', 'plane0_file']:
                full_file_path = os.path.join(directory_path, df.iloc[i][imName])
                if os.path.exists(full_file_path):
                    sparse_tensor, memory_mb = getSize(full_file_path)
                    sizeFull += memory_mb
                    df.at[i, f"{imName.split('_')[0]}_sparse_tensor"] = sparse_tensor
                else:
                    print(f"Image not found: {full_file_path}")

        # Save the modified dataframe to a pickle file
        output_file_path = os.path.join(directory_path, f"processed_{filename.replace('.csv', '.pkl')}")
        with open(output_file_path, 'wb') as f:
            pickle.dump(df, f)

        print(f"Processed file pickled: {output_file_path}")

Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 0
0
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 100
70.01553344726562
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 200
140.7273759841919
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 300
209.30729579925537
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 400
280.94514560699463
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 500
351.8641185760498
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 600
426.20957946777344
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 700
495.62755393981934
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 800
565.2263946533203
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 900
633.3639249801636
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 1000
703.8807964324951
Processing file: parsed_data_bnb_WithWire_04.csv, iteration: 1100
773.9233236312866
Processed file p

In [ ]:
import os
import pickle
import pandas as pd

# Directory containing the pickled files
directory_path = '/workspace/outputDir'

# List to store individual DataFrames
dataframes = []

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.pkl'):  # Check if the file is a pickle file
        print(f"Loading file: {filename}")
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'rb') as f:
            df = pickle.load(f)  # Load the pickled DataFrame
            dataframes.append(df)  # Append to the list

# Combine all DataFrames into one
combined_dataset = pd.concat(dataframes, ignore_index=True)

print("All pickled datasets loaded and combined successfully!")

Loading file: processed_parsed_data_bnb_WithWire_04.pkl
Loading file: processed_parsed_data_bnb_WithWire_01.pkl
Loading file: processed_parsed_data_bnb_WithWire_05.pkl
Loading file: processed_parsed_data_bnb_WithWire_06.pkl
Loading file: processed_parsed_data_bnb_WithWire_03.pkl
Loading file: processed_parsed_data_bnb_WithWire_02.pkl
Loading file: processed_parsed_data_bnb_WithWire_08.pkl
Loading file: processed_parsed_data_bnb_WithWire_07.pkl
Loading file: processed_parsed_data_bnb_WithWire_11.pkl
Loading file: processed_parsed_data_bnb_WithWire_12.pkl
Loading file: processed_parsed_data_bnb_WithWire_10.pkl
Loading file: processed_parsed_data_bnb_WithWire_13.pkl
Loading file: processed_parsed_data_bnb_WithWire_16.pkl
Loading file: processed_parsed_data_bnb_WithWire_14.pkl
Loading file: processed_parsed_data_bnb_WithWire_17.pkl
Loading file: processed_parsed_data_bnb_WithWire_00.pkl
Loading file: processed_parsed_data_nue_WithWire_01.pkl
Loading file: processed_parsed_data_nue_WithWire

In [7]:
for i in range(len(combined_dataset['plane2_sparse_tensor'])):
    combined_dataset['plane2_sparse_tensor'][i].to_dense()

In [10]:


# Example usage
file_name = combined_dataset.iloc[1300]['plane1_file']

# Join the directory path with the file name
full_file_path = os.path.join(directory_path, file_name)

print(full_file_path)

/workspace/outputDir/7017_1062_53144_1.png


In [11]:
from PIL import Image
import numpy as np

# Load the image using Pillow
image = Image.open(full_file_path)

# Convert the image to grayscale
image = image.convert('L')

# Convert the grayscale image to a NumPy array
image_array = np.array(image)

image_array = 255 - image_array



print(image_array.shape)  # Print the shape of the grayscale image array

(1067, 2400)


In [12]:
image_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [52]:
image_array = 255 - image_array

In [13]:
import torch

# Convert the NumPy array to a PyTorch tensor
dense_tensor = torch.tensor(image_array, dtype=torch.uint8)

# Find the indices of non-zero elements
non_zero_indices = torch.nonzero(dense_tensor, as_tuple=True)

# Get the values of the non-zero elements
non_zero_values = dense_tensor[non_zero_indices]

# Create a sparse tensor
sparse_tensor = torch.sparse_coo_tensor(
    indices=torch.stack(non_zero_indices),
    values=non_zero_values,
    size=dense_tensor.shape
)


print(sparse_tensor)

tensor(indices=tensor([[   0,    0,    0,  ..., 1066, 1066, 1066],
                       [1177, 1178, 1179,  ..., 1999, 2000, 2001]]),
       values=tensor([255, 255, 255,  ...,  82,  16,  12]),
       size=(1067, 2400), nnz=12017, dtype=torch.uint8, layout=torch.sparse_coo)


In [14]:
# Coalesce the sparse tensor to ensure it is in a valid state
sparse_tensor = sparse_tensor.coalesce()

# Calculate memory usage of the sparse tensor
indices_memory = sparse_tensor.indices().numel() * sparse_tensor.indices().element_size()
values_memory = sparse_tensor.values().numel() * sparse_tensor.values().element_size()
sparse_tensor_memory_bytes = indices_memory + values_memory
sparse_tensor_memory_mb = sparse_tensor_memory_bytes / (1024 ** 2)

# Calculate memory usage of the dense tensor
dense_tensor_memory_bytes = dense_tensor.numel() * dense_tensor.element_size()
dense_tensor_memory_mb = dense_tensor_memory_bytes / (1024 ** 2)

# Print the comparison
print(f"Memory used by sparse_tensor: {sparse_tensor_memory_mb:.2f} MB")
print(f"Memory used by dense_tensor: {dense_tensor_memory_mb:.2f} MB")

Memory used by sparse_tensor: 0.19 MB
Memory used by dense_tensor: 2.44 MB
